In [ ]:
!pip -q install googletrans

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import re
from bs4 import BeautifulSoup
from tqdm import tqdm, tqdm_gui
tqdm.pandas(ncols=75) 

from googletrans import Translator
from dask import bag, diagnostics

In [ ]:
train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")
test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

In [ ]:
def remove_space(text):
    return " ".join(text.split())

def remove_punctuation(text):
    return re.sub("[!@#$+%*:()'-]", ' ', text)

def remove_html(text):
    soup = BeautifulSoup(text, 'lxml')
    return soup.get_text()

def remove_url(text):
    return re.sub(r"http\S+", "", text)

def translate(text):
    translator = Translator()
    return translator.translate(text, dest='en').text

def clean_text(text):
    text = remove_space(text)
    text = remove_html(text)
    text = remove_url(text)
    text = remove_punctuation(text)
    return text

In [ ]:
train['premise'] = train.premise.progress_apply(lambda text : clean_text(text))
train['hypothesis'] = train.hypothesis.apply(lambda text : clean_text(text))

In [ ]:
eng = train.loc[train.lang_abv == "en"].copy()

In [ ]:
def translate_parallel(df):
    premise_bag = bag.from_sequence(df.premise.tolist()).map(translate)
    hypothesis_bag = bag.from_sequence(df.hypothesis.tolist()).map(translate)
    with diagnostics.ProgressBar():
        premises = premise_bag.compute()
        hypothesis = hypothesis_bag.compute()
    df[['premise', 'hypothesis']] = list(zip(premises, hypothesis))
    return df

In [ ]:
non_eng =  train.loc[train.lang_abv != "en"].copy().pipe(translate_parallel)

In [ ]:
train_translated = eng.append(non_eng)

In [ ]:
train_translated.head()

In [ ]:
train.shape

In [ ]:
train_translated.shape

In [ ]:
train_translated.head()

In [ ]:
train_translated.to_csv("train_english_translated.csv", index=False)

In [ ]:
test_eng = test.loc[test.lang_abv == "en"].copy()

In [ ]:
test_non_eng = test.loc[test.lang_abv != "en"].copy().pipe(translate_parallel)

In [ ]:
test_translated = test_eng.append(test_non_eng)

In [ ]:
print(test_translated.shape)
print(test.shape)

In [ ]:
test_translated.to_csv("test_english_translated.csv", index=False)

In [ ]:
test_translated.tail()